In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from scipy.spatial import KDTree
import numpy as np
import pandas as pd

y_true = np.load('/Users/xiatianqi/Downloads/y_true.npy')
y_pred = np.load('/Users/xiatianqi/Downloads/y_pred.npy')

y_true_one_day = np.moveaxis(y_true[4],-1,0)[0]
y_pred_one_day = np.moveaxis(y_pred[4],-1,0)[0]

def probalistic_cross_k_function(y_true,y_pred,minkov_degree=1,max_distance=10):
    y_true_list = np.stack(np.where(y_true == 1)).T
    y_true_kdtree = KDTree(y_true_list)
    
    y_pred_list = np.stack(np.where(y_pred >= 0)).T
    y_pred_kdtree = KDTree(y_pred_list)
    
    dense = np.mean(y_pred_one_day)
    
    k_value_list = []
    random_list = []
    
    for i in range(max_distance+1):
        total_result = y_true_kdtree.sparse_distance_matrix(y_pred_kdtree,p=minkov_degree,max_distance=i)
        y_list = []
        y_random_list = []
        for j in range(len(y_true_list)):
            neighbor_obj = total_result.getrow(j).indices
            final_list = y_pred_list[list(neighbor_obj)].T
            y_list.append(sum(y_pred_one_day[final_list[0],final_list[1]]))
            y_random_list.append(len(neighbor_obj)*dense)
        k_value_list.append(mean(y_list) / dense)
        random_list.append(mean(y_random_list) / dense)
    
    return k_value_list,random_list

df = pd.DataFrame(probalistic_cross_k_function(y_true_one_day,y_pred_one_day,minkov_degree=1,max_distance=7))

df.index = ['kf value','random kf value']
df = df.stack().reset_index()

df.columns = ['type','distance','kf_value']
ax = sns.lineplot(x='distance',y='kf_value',hue='type',data=df)
plt.show()